In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np
import cmdstanpy ## import stan interface for Python
from scipy.integrate import solve_ivp
from matplotlib.gridspec import GridSpec
import os

import sys
sys.path.append("..")

import stancourse.utilities as util

if os.name == "nt": ## adds compiler to path in Windows
    cmdstanpy.utils.cxx_toolchain_path() 

# Solutions to exercises

## Exercise 1: User-defined distribution

Let $E_1, E_2, \dots, E_n$ be independent, exponentially distributed random variables with rates $a_1, a_2,\dots, a_n$ respectively. The sum $X = E_1 + E_2 + \cdots + E_n$ has a so-called *hypoexponential* distribution. A special case is the Erlang distribution, in which case $a_1 = a_2 = \cdots = a_n$. Stan does not provide the hypoexponential distribution, and so we have to implement this ourselves. On [wikipedia](https://en.wikipedia.org/wiki/Hypoexponential_distribution), we find that the PDF of $X$ is given by 

\begin{equation}
 f_X(t) = - \alpha \exp(t Q) Q 1
\end{equation}
where $\alpha = (1, 0, 0, \dots, 0) \in \mathbb{R}^n$ is a row vector, and $1$ is a vector of $n$ ones $(1, 1, \dots, 1)^T \in \mathbb{R}^n$. The matrix $Q$ is given by
\begin{equation}
 Q = \left(\begin{array}{ccccc}
 -a_1 & a_1 & 0 & \cdots & 0\\
 0 & -a_2 & a_2 & \cdots & 0 \\
 \vdots & \ddots & \ddots & \ddots & \vdots \\
 0 & \cdots & 0 & -a_{n-1} & a_{n-1} \\
 0 & \cdots & 0 & 0 & -a_n
 \end{array}\right)
\end{equation}
The function $\exp$ denotes matrix exponentiation and is available in Stan as `matrix_exp`.

In [ ]:
util.show_stan_model("../stan-models/hypoexp.stan")

In [ ]:
sm = cmdstanpy.CmdStanModel(stan_file="../stan-models/hypoexp.stan")
data_dict = {
    "n" : 4,
    "a" : [0.2,1,2.5,3.1],
}
sam = sm.sample(chains=1, iter_sampling=10000, data=data_dict)

fig, ax = plt.subplots(1, 1, figsize=(10,3))
ts = sam.stan_variable("t")
xs = sam.stan_variable("x")
bins = ax.hist([ts, xs], bins=50, label=["t", "x"], density=True)
ax.legend(); ax.set_ylabel("density")

## Exercise 2: Classification in a mixture model

Example application: seroprevalence data

* $X_1, X_2, \dots, X_N$ (properly transformed) antibody titers
* With probability $p$, subject $i$ is "positive", and "negative" otherwise. 
* postive and negative titers have normal distribution with means $\mu_1 < \mu_2$ and standard deviations $\sigma_1$ and $\sigma_2$. 

\begin{equation}
    X_i \sim \left\{\begin{array}{ll}
        \mathcal{N}(\mu_1, \sigma_1) & \mbox{if $i$ negative} \\
        \mathcal{N}(\mu_2, \sigma_2) & \mbox{if $i$ positive}
    \end{array}\right.
\end{equation}

* We don't know the status of each individual, but only the titer $X_i$

**Classification of status**
Let $I_i \in \{ pos, neg \}$ denote the status of subject $i$. Bayes rule says
\begin{equation}
    \mathbb{P}(I_i = pos | X_i) = \frac{p L(X_i | pos)}{p L(X_i | pos) + (1-p) L(X_i | neg)}
\end{equation}
Note that $I_i$ is the "parameter", $L(X_i | pos)$ is the PDF of $\mathcal{N}(\mu_2, s\sigma_2)$,
and the prior of $s_i$ is given by
\begin{equation}
\pi(pos) = p\,,\quad \pi(neg) = 1-p
\end{equation}

In [ ]:
util.show_stan_model("../stan-models/mixture_classification.stan")

In [ ]:
p = 0.3
mu1, mu2 = -1, 2
sigma1, sigma2 = 0.5, 1.0

N = 1000
I = sts.bernoulli.rvs(p, size=N)
X = [sts.norm.rvs(loc=mu1, scale=sigma1) if i == 0 
     else sts.norm.rvs(loc=mu2, scale=sigma2) for i in I]

In [ ]:
sm = cmdstanpy.CmdStanModel(stan_file="../stan-models/mixture_classification.stan")
data_dict = {"N" : N, "X" : X}
sam = sm.sample(data=data_dict, chains=1)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(7,4))

axs[0].hist(X, 50, density=True, label="data")
axs[0].set_xlabel("titer $X$")
axs[0].set_ylabel("density")

mu_est = sam.stan_variable("mu")
sigma_est = sam.stan_variable("sigma")
p_est = sam.stan_variable("p")
xs = np.linspace(np.min(X), np.max(X), 1000)
y1s = sts.norm.pdf(xs, loc=np.mean(mu_est[:,0]), scale=np.mean(sigma_est[:,0]))
y2s = sts.norm.pdf(xs, loc=np.mean(mu_est[:,1]), scale=np.mean(sigma_est[:,1]))

p_mean = np.mean(p_est)
axs[0].plot(xs, (1-p_mean) * y1s, linewidth=3, label="negative")
axs[0].plot(xs, p_mean * y2s, linewidth=3, label="positive")

axs[0].legend()

ppos = np.mean(sam.stan_variable("ppos"), axis=0)

axs[1].scatter(ppos, I, marker='|', color='k')
axs[1].set_xlim(-0.05, 1.05)
axs[1].set_ylim(-0.1, 1.1)

axs[1].set_yticks([0,1])
axs[1].set_yticklabels(["neg", "pos"])
axs[1].set_xlabel("classification")

In [ ]:
fig ## data, fit and classification